In [4]:
import json
def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

In [5]:
def get_feature_set(json_data):
    feature_set = set()
    for occupation, info in json_data.items():
        for feature in info['attribute']:
            # print(feature)
            feature_set.add(feature)
    return feature_set

data = load_data('output/occupation_attribute_100.json')
feature_set = get_feature_set(data)


This doesn't work because it's only looking at sentence level.

In [6]:
# import os
# import numpy as np
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize import word_tokenize
# from collections import defaultdict, Counter
# import nltk
# from tqdm import tqdm

# # Ensure required NLTK resources are downloaded
# nltk.download('punkt')  # Tokenizer
# nltk.download('wordnet')  # Lemmatization data

# def load_and_process_sentences(files):
#     sentences = []
#     for file_path in files:
#         with open(file_path, 'r', encoding='utf-8') as file:
#             sentences.extend(file.readlines())
#     return sentences

# def build_probability_matrix(sentences, feature_set):
#     lemmatizer = WordNetLemmatizer()
#     feature_counts = Counter()
#     co_occurrence_counts = defaultdict(Counter)
    
#     for sentence in tqdm(sentences, desc="Processing Sentences"):
#         words = word_tokenize(sentence.lower())
#         lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word.isalpha()]
#         features_in_sentence = [word for word in lemmatized_words if word in feature_set]
        
#         # Track seen features in this sentence to prevent double counting in the context
#         seen_features = set()
        
#         for i, feature in enumerate(features_in_sentence):
#             if feature not in seen_features:
#                 feature_counts[feature] += 1
#                 seen_features.add(feature)
            
#             # Count only subsequent features different from the current feature
#             for subsequent_feature in features_in_sentence[i+1:]:
#                 if subsequent_feature != feature:
#                     co_occurrence_counts[feature][subsequent_feature] += 1

#     feature_list = sorted(feature_set)
#     matrix_size = len(feature_list)
#     probability_matrix = np.zeros((matrix_size, matrix_size))
#     feature_index = {feature: idx for idx, feature in enumerate(feature_list)}
    
#     for f1 in feature_list:
#         f1_idx = feature_index[f1]
#         for f2 in feature_list:
#             f2_idx = feature_index[f2]
#             if feature_counts[f1] > 0:
#                 probability_matrix[f1_idx][f2_idx] = co_occurrence_counts[f1][f2] / feature_counts[f1]

#     return feature_list, probability_matrix, feature_counts, co_occurrence_counts




In [7]:
# dataset_dir = 'wikipedia-biography-dataset/wikipedia-biography-dataset'
# subsets = ['train/train']#, 'valid/valid', 'test/test']
# sent_files = [os.path.join(dataset_dir, f"{subset}.sent") for subset in subsets]
# sentences = load_and_process_sentences(sent_files)

# feature_list, probability_matrix, feature_counts, co_occurrence_counts = build_probability_matrix(sentences, feature_set)

# # print("Feature List:", feature_list)
# # print("Probability Matrix:\n", prob_matrix)


# from wikipedia

In [28]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from tqdm import tqdm
import numpy as np

def build_probability_matrix_from_list(data, feature_set):
    lemmatizer = WordNetLemmatizer()
    feature_counts = Counter()
    co_occurrence_counts = defaultdict(lambda: defaultdict(int))
    
    # Process each occupation group in the data
    for summary in tqdm(data, desc="Processing occupations"):
        words = word_tokenize(summary.lower())
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word.isalpha()]
        features_in_summary = [word for word in lemmatized_words if word in feature_set]

        in_this_summary = set()

        for i in range(len(features_in_summary)):
            for j in range(i + 1, len(features_in_summary)):
                if (features_in_summary[i],features_in_summary[j]) not in in_this_summary:
                    in_this_summary.add((features_in_summary[i], features_in_summary[j]))
                    feature_counts[features_in_summary[i]] += 1
                    co_occurrence_counts[features_in_summary[i]][features_in_summary[j]] += 1

    feature_list = sorted(feature_set)
    matrix_size = len(feature_list)
    probability_matrix = np.zeros((matrix_size, matrix_size))
    
    for i, f1 in enumerate(feature_list):
        total_co_occurrences = sum(co_occurrence_counts[f1].values())
        for j, f2 in enumerate(feature_list):
            if f1 != f2 and total_co_occurrences > 0:
                probability_matrix[i, j] = co_occurrence_counts[f1][f2] / total_co_occurrences

    return feature_list, probability_matrix, feature_counts, co_occurrence_counts



In [29]:
# json_file = 'output/occupation_data_100.json'
json_file = 'output/occupation_summaries_200.json'
data = load_data(json_file)
wiki_summary_list = [i for i in data.values()]
feature_list, prob_matrix, feature_counts, co_occurrence_counts = build_probability_matrix_from_list(wiki_summary_list, feature_set)

Processing occupations: 100%|██████████| 105840/105840 [01:47<00:00, 983.23it/s] 


In [10]:
import numpy as np

def save_outputs(feature_list, probability_matrix, feature_counts, co_occurrence_counts, output_prefix):
    # Save the feature list
    with open(f'{output_prefix}_feature_list.json', 'w') as f:
        json.dump(feature_list, f, indent=4)
    
    # Save the probability matrix using NumPy's save function
    np.save(f'{output_prefix}_probability_matrix.npy', probability_matrix)
    
    # Save the feature counts
    with open(f'{output_prefix}_feature_counts.json', 'w') as f:
        json.dump(dict(feature_counts), f, indent=4)
    
    # Save the co_occurrence counts
    with open(f'{output_prefix}_co_occurrence_counts.json', 'w') as f:
        # Convert defaultdict to a normal dictionary for JSON serialization
        co_occurrence_dict = {k: dict(v) for k, v in co_occurrence_counts.items()}
        json.dump(co_occurrence_dict, f, indent=4)

save_outputs(feature_list, prob_matrix, feature_counts, co_occurrence_counts, 'output/wiki')

In [11]:
def save_sorted_probabilities(feature_list, probability_matrix, feature_counts, co_occurrence_counts, output_file):
    feature_index = {feature: idx for idx, feature in enumerate(feature_list)}
    sorted_probabilities = []

    # Collect all relevant data
    for f1 in feature_list:
        for f2 in feature_list:
            if f1 != f2:
                f1_idx = feature_index[f1]
                f2_idx = feature_index[f2]
                prob = probability_matrix[f1_idx][f2_idx]
                f1_count = feature_counts[f1]
                co_occurrence = co_occurrence_counts[f1][f2]
                sorted_probabilities.append(((f1, f2), prob, f1_count, co_occurrence))

    # Sort by probability, descending
    sorted_probabilities.sort(key=lambda x: x[1], reverse=True)

    # Save to file
    with open(output_file, 'w') as file:
        for entry in sorted_probabilities:
            line = f"{entry[0]}: Probability={entry[1]:.4f}, Count of {entry[0][0]}={entry[2]}, Count of {entry[0][0]} followed by {entry[0][1]}={entry[3]}\n"
            file.write(line)


In [12]:
output_file = 'wiki_sorted_probabilities_report.txt'
save_sorted_probabilities(feature_list, prob_matrix, feature_counts, co_occurrence_counts, output_file)

# from generation

In [30]:
import json
from pathlib import Path

def load_data_from_jsonl(jsonl_files):
    data = []
    for file_path in jsonl_files:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # Each line is a complete JSON object
                entry = json.loads(line)
                data.append(entry)
    return data

labeled_path = Path('data/unlabeled')
jsonl_files = list(labeled_path.glob('*.jsonl'))
all_data = load_data_from_jsonl(jsonl_files)
gen_output_list = [i['output'] for i in all_data]


In [31]:
feature_list_gen, prob_matrix_gen, feature_counts_gen, co_occurrence_counts_gen = build_probability_matrix_from_list(gen_output_list, feature_set)

Processing occupations: 100%|██████████| 6000/6000 [00:07<00:00, 818.90it/s] 


In [32]:
save_outputs(feature_list_gen, prob_matrix_gen, feature_counts_gen, co_occurrence_counts_gen, 'output/gen')

In [33]:
output_file = 'gen_sorted_probabilities_report.txt'
save_sorted_probabilities(feature_list_gen, prob_matrix_gen, feature_counts_gen, co_occurrence_counts_gen, output_file)

# compare

In [41]:
def compare_probability_matrices(wiki_matrix, gen_matrix, feature_list, output_file):

    matrix_size = len(feature_list)
    differences = np.zeros((matrix_size, matrix_size))
    
    for i in range(matrix_size):
        for j in range(matrix_size):
            if i != j:  # We skip comparing a feature with itself
                differences[i][j] = wiki_matrix[i][j] - gen_matrix[i][j]

    # print(differences)

    flat_differences = []
    for i in range(matrix_size):
        for j in range(matrix_size):
            if i != j:
                flat_differences.append(((feature_list[i], feature_list[j]), differences[i][j]))

    flat_differences.sort(key=lambda x: x[1], reverse=True)
    
    significant_differences = {
        'most_positive': flat_differences[:20], 
        'most_negative': flat_differences[-20:] 
    }

    with open(output_file, 'w') as f:
        json.dump(significant_differences, f, indent=4)

    return significant_differences

In [42]:
print(len(prob_matrix))
print(len(feature_list))

443
443


In [43]:

wiki_prob_matrix = prob_matrix
gen_prob_matrix = prob_matrix_gen
feature_list = feature_list

output_file_path = 'comparison_results.json'
comparison_results = compare_probability_matrices(wiki_prob_matrix, gen_prob_matrix, feature_list, output_file_path)
